#### Section 1.4.1 Voltage Dividers
Taken from "Switch-Mode Power Supplies" by Christophe P. Basso

#### Imports

In [1]:
from pathlib import Path
import py4spice as spi

#### Path to Ngspice and the project

In [2]:
# Path to NGSPICE
NGSPICE_EXE = Path("/usr/bin/ngspice")

# Path to my project
PROJ_PATH = Path("/workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers")

#### Define simulation results directory

In [3]:
# Create sim_results directory if doesn't exist
RESULTS_PATH = PROJ_PATH / "sim_results"
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

#### Define netlists and create dut.cir from modified raw_kicad.cir

In [4]:
# Create netlists directory and files if they don't exist
NETLISTS_PATH = PROJ_PATH / "netlists"
NETLISTS_PATH.mkdir(parents=True, exist_ok=True)  # Create the directory

# Create title.cir with content only if it doesn't exist
TITLE_FILENAME = NETLISTS_PATH / "title.cir"
if not TITLE_FILENAME.exists():
    with open(TITLE_FILENAME, "w") as f:
        f.write("* First line in netlist must be a comment")
title = spi.Netlist(NETLISTS_PATH / TITLE_FILENAME)

# These netlist objects will be combined and a top netlist will be created later
stimulus = spi.Netlist(NETLISTS_PATH / "stimulus.cir")
supplies = spi.Netlist(NETLISTS_PATH / "supplies.cir")

# Create "end" netlist object, but no need to write to a file
end = spi.Netlist()
end.insert_line(0, ".end")

# these are two files the program will create later
TOP_FILENAME = NETLISTS_PATH / "top.cir"
CONTROL_FILENAME = NETLISTS_PATH / "control.cir"

# Create dut.cir from raw_kicad.cir
raw_kicad = spi.Netlist(NETLISTS_PATH / "raw_kicad.cir")
raw_kicad.del_line_starts_with(".title")  # delete first line (title)
raw_kicad.del_line_starts_with(".end")  # delete last line (.end)
raw_kicad.del_slash()  # delete forward slashes from node names
dut = raw_kicad
dut.write_to_file(NETLISTS_PATH / "dut.cir")  # this is now the dut (device under test)

#### Create signal vectors

In [5]:
VEC_ALL = spi.Vectors("all")
VEC_ALL_EXPANDED = spi.Vectors(
    "in out1 out1_meas out2 out2_meas vee vee#branch vin#branch vmeas1#branch vmeas2#branch"
)
VEC_VIN = spi.Vectors("in")
VEC_IIN = spi.Vectors("vin#branch")
VEC_VOUT1 = spi.Vectors("out1")
VEC_VOUT2 = spi.Vectors("out2")
VEC_IOUT1 = spi.Vectors("vmeas1#branch")
VEC_IOUT2 = spi.Vectors("vmeas2#branch")
VEC_IN_OUT = VEC_VIN + VEC_VOUT1 + VEC_VOUT2

#### Define analyses

In [6]:
list_of_analyses: list[spi.Analyses] = [] # create empty list. Next sections define

# 1st analysis: operating point
# for this simulation, we'll have only one analysis
op_cmd = "op"
op1 = spi.Analyses("op1", "op", op_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses.append(op1)

#### Create control section

In [7]:
my_control = spi.Control()  # create 'my_control' object
my_control.insert_lines(["listing"])  # cmd to list out netlist
for analysis in list_of_analyses:  # statements for all analyses
    my_control.insert_lines(analysis.lines_for_cntl())
spi.print_section("Control File", my_control)  # print out contents
my_control.content_to_file(CONTROL_FILENAME)  # creat the actual file
control = spi.Netlist(NETLISTS_PATH / CONTROL_FILENAME) # create netlist object


--- Control File ---
.control
* Timestamp: Sun Mar  3 05:23:39 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/sim_results/op1.txt
quit
.endc
--------------------



#### Simulate

In [8]:
# Combine the netlists and write out into one top netlist, ready to simulate
top = title + dut + supplies + stimulus + control + end

top.write_to_file(TOP_FILENAME)
spi.print_section("top netlist", top)

# prepare simulate object, print out command, and simulate
sim1 = spi.Simulate(NGSPICE_EXE, TOP_FILENAME)
spi.print_section("Ngspice Command", sim1)
sim1.run() # run the Ngspice simulation


--- top netlist ---
* voltage divider, section 1.4.1
r1 in out1 140
r2 in out2 132
rload1 out1 out1_com 100
rload2 out2 out2_com 50
vmeas1 out1_com 0 dc 0
vmeas2 out2_com 0 dc 0
vin in 0 dc 12
.control
* timestamp: sun mar  3 05:23:39 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/sim_results/op1.txt
quit
.endc
.end
-------------------


--- Ngspice Command ---
/usr/bin/ngspice -b /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/netlists/top.cir
-----------------------


Note: No compatibility mode selected!


Circuit: * voltage divider, section 1.4.1

	* voltage divider, section 1.4.1

     2 : .global gnd
     3 : r1 in out1 140
     4 : r2 in out2 132
     5 : rload1 out1 out1_com 100
     6 : rload2 out2 out2_com 50
     7 : vmeas1 out1_com 0 dc 0
     8 : vmeas2 out2_com 0 dc 0
     9 : vin in 0 dc 12
    10 : .control
    12 : 

Using SPARSE 1.3 as Direct Linear Solver


#### Gather simulation results and transform into more usable format

In [9]:
my_sim_results = [
    spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses]

#### Calculations and printout

In [10]:
# 1st (and only) annlysis was operating point
my_op1 = my_sim_results[0]

# Extract the values from the op analysis
vin: float = my_op1.table_value(str(VEC_VIN))
iin: float = my_op1.table_value(str(VEC_IIN))
vout1: float = my_op1.table_value(str(VEC_VOUT1))
vout2: float = my_op1.table_value(str(VEC_VOUT2))
iout1: float = my_op1.table_value(str(VEC_IOUT1))
iout2: float = my_op1.table_value(str(VEC_IOUT2))

# Calculate power and efficiency
pr1: float = (vin - vout1) * iout1
pr2: float = (vin - vout2) * iout2
pout1: float = vout1 * iout1
pout2: float = vout2 * iout2
pout: float = pout1 + pout2
pin: float = vin * -iin
eta: float = pout / pin

print(f"p1 = {pr1:.4g}W")
print(f"p2 = {pr2:.4g}W")
print(f"pout1 = {pout1:.4g}W")
print(f"pout2 = {pout2:.4g}W")
print(f"pout = {pout:.4g}W")
print(f"pin = {pin:.4g}W")
print(f"eta = {eta*100:.4g}%")

p1 = 0.35W
p2 = 0.5738W
pout1 = 0.25W
pout2 = 0.2174W
pout = 0.4674W
pin = 1.391W
eta = 33.59%
